In [93]:
import pandas as pd
import numpy as np

In [95]:
path = "../../../dataVault/labour_report_ds/Jan_2021.csv"
df = pd.read_csv(path, dtype={"Customer number": np.float64 ,"Schd Time Start": str, "PO": str})
# df['Customer number'] =
# df = df.round({'Customer number' : 3})
df['Customer number'] = df['Customer number'].round(3)

In [96]:
others = df[df['Customer number'].isin([3763,3268,3483,2317,4236,3476,4156,3966])]
df_4138 = df.loc[(df['Customer number'] > 4138.000) & (df['Customer number'] <= 4138.1)]
df_1021 = df.loc[(df['Customer number'] > 1021.000) & (df['Customer number'] <= 1021.1)]

keys_1021 = df_1021.groupby('Waste Type').indices.keys()
keys_4138 = df_4138.groupby('Waste Type').indices.keys()
keys_other = others.groupby('Waste Type').indices.keys()

# print(f"4138 - {keys_4138} 1021 - {keys_1021} Others - {keys_other}")

# testdf



dict_keys(['Cardboard', 'Comingle', 'Dry Waste', 'Food Waste', 'General Waste', 'Organics', 'Paper/Cardboard'])

In [106]:
# df['col_3'] = df.apply(lambda x: f(x.col_1, x.col_2), axis=1)
df_4138_group = df_4138.groupby('Customer number')
df_4138_group_keys = df_4138_group.indices.keys()
# test df as 4138.003

testdf = df_4138_group.get_group(4138.003)
# dict_keys([4138.003, 4138.004, 4138.005, 4138.006, 4138.007, 4138.008])

df_4138_group_keys
testdf['Weight'].dtype


testb = testdf.head(1)
testb['Weight'][457] == 0

True

In [98]:
# if weight is equal to zero 
# Waste Standard Density
waste_sd = {
    "General Waste" : {
    "0.12" : 12.6,
    "0.24" : 25.2,
    "0.66" : 69.3,
    "1.1" : 115.5,
    "1.5" : 157.5,
    "3" : 315.0,
    "4.5" : 472.5
},
"Dry Waste" : {
    "0.12" : 8.4,
    "0.24" : 16.8,
    "0.66" : 46.2,
    "1.1" : 77,
    "1.5" : 105,
    "3" : 210,
    "4.5" : 315
},
"Comingle" : {
    "0.12" : 7.2,
    "0.24" : 14.4,
    "0.66" : 39.6,
    "1.1" : 66.0,
    },
 "Paper/Cardboard":{
     "0.12" : 6,
    "0.24" : 12,
    "0.66" : 33,
    "1.1" : 55,
    "1.5" : 75,
    "3" : 150,
    "4.5" : 225
 },
 "Paper":{
    "0.12" : 10.8,
    "0.24" : 21.6,
    "0.66" : 59.4
 },
"Glass" : {
    "0.12" : 24,
    "0.24" : 48,
    "0.66" : 132
},
"Organics" : {
    "0.12" : 24,
    "0.24" : 48
}
}

In [99]:
# df['col_3'] = df.apply(lambda x: f(x.col_1, x.col_2), axis=1)
def cal_weight(status, waste_type, qty_serviced, bin_volume):
    
    if status == 'C':
        
#         waste type =>  Food Waste => Organics / Cardboard => Paper/Cardboard
        if waste_type == 'Food Waste':
            return qty_serviced * waste_sd['Organics'][str(bin_volume)]
    
        elif waste_type == 'Cardboard':
            return qty_serviced*waste_sd['Paper/Cardboard'][str(bin_volume)]
            
        return qty_serviced*waste_sd[waste_type][str(bin_volume)]
    else:
        return np.nan

testdf['Total weight picked up - kg (if available)'] = (testdf
                                                        .apply(
                                                            lambda df: cal_weight(
                                                                df['Status'],
                                                                df['Waste Type'],
                                                                df['Qty Serviced'],
                                                                df['Bin Volume']
                                                            ), 
                                                            axis=1
                                                        ))

#  Weight Channel
# a = list(waste_sd["Organics"].keys())
# b = str(testdf["Bin Volume"][457])
# b == a[1]


<ipython-input-99-842e53c45b3e>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf['Total weight picked up - kg (if available)'] = (testdf


In [100]:
# if weight is 0 then transform data to 
# Total weight picked up - kg (if available)


# Tipping Site
#     General waste 
#     Velio Waste and Recycling

# Comingle
#     Visy Recycling
#     PolyTrade Recycling

# Cardboard 
#     Grima Recycling
#      PolyTrade Recycling

# 'Dry Waste'
# Velia

# Food Waste 
#     Velia 

# dict_keys([
# 'Cardboard', 
# 'Paper/Cardboard'

# 'Comingle', 'Dry Waste', 'Food Waste', 'General Waste', 'Organics', ])

def facility(waste_type):
    facility = {
        'General Waste' : 'Veolia Waste Centre Clyde',
        'Dry Waste' : 'Veolia Waste Centre Clyde',
        'Food Waste' : 'Eathpower Technologies',
        'Organics' : 'Eathpower Technologies',
        'Paper/Cardboard' : 'Grima Recycling',
        'Cardboard' : 'Grima Recycling',
        'Comingle' : 'Visy Recycling'
    }
    return facility[waste_type]
testdf["Processing Facility sent to (Optional)"] = testdf['Waste Type'].transform(facility)

<ipython-input-100-6341e2307b43>:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf["Processing Facility sent to (Optional)"] = testdf['Waste Type'].transform(facility)


In [101]:
# report_col_df = testdf[['Date', 'Address 1',  'Waste Type', 'Bin Volume','Qty Serviced', 'Status']]
# testdf = testdf[testdf["Status"].isin(['C'])]

# df = df.rename(columns={'oldName1': 'newName1', 'oldName2': 'newName2'})
#  df["D"] = np.nan

# testdf["Dock"] = np.nan
testdf["Equipment"] = "Mobile bin"
testdf["Bin Rejected due to contamination (Optional)"] = np.nan

testdf = (testdf
 .rename(columns={
     'Date' : 'Pick-up Date (dd/mm/yyyy)',
     'Address 1': 'Site Address',
     'Bin Volume' : 'Size (m3)',
     'Qty Serviced' : 'Units Collected'
     })
)

testdf = (testdf[
    ['Pick-up Date (dd/mm/yyyy)',
     'Site Address',
     'Waste Type',
     'Equipment',
     'Size (m3)',
     'Units Collected',
     'Total weight picked up - kg (if available)',
     'Processing Facility sent to (Optional)',
     'Bin Rejected due to contamination (Optional)'
    ]])

testdf.to_csv("./sample_labour_report.csv", columns=[
    'Pick-up Date (dd/mm/yyyy)',
     'Site Address',
     'Dock',
     'Waste Type',
     'Equipment',
     'Size (m3)',
     'Units Collected',
     'Total weight picked up - kg (if available)',
     'Processing Facility sent to (Optional)',
     'Bin Rejected due to contamination (Optional)'
    ], index=False)


testdf
# df.assign(C="",D=np.nan)
# r_report_col_df.assign(Dock=np.nan) 
# testdf['Status'].filter("F")


<ipython-input-101-cd14f36c6ced>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf["Dock"] = np.nan
<ipython-input-101-cd14f36c6ced>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf["Equipment"] = "Mobile bin"
<ipython-input-101-cd14f36c6ced>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

,Pick-up Date (dd/mm/yyyy),Site Address,Dock,Waste Type,Equipment,Size (m3),Units Collected,Total weight picked up - kg (if available),Processing Facility sent to (Optional),Bin Rejected due to contamination (Optional)
457,30/01/21,14-18 Lee St,NaN,General Waste,Mobile bin,0.24,1,25.2,Veolia Waste Centre Clyde,NaN
5280,27/01/21,14-18 Lee St,NaN,General Waste,Mobile bin,0.24,1,25.2,Veolia Waste Centre Clyde,NaN
5281,27/01/21,14-18 Lee St,NaN,Comingle,Mobile bin,0.24,1,NaN,Visy Recycling,NaN
10479,23/01/21,14-18 Lee St,NaN,General Waste,Mobile bin,0.24,3,75.6,Veolia Waste Centre Clyde,NaN
15124,20/01/21,14-18 Lee St,NaN,Comingle,Mobile bin,0.24,1,14.4,Visy Recycling,NaN
15125,20/01/21,14-18 Lee St,NaN,General Waste,Mobile bin,0.24,1,25.2,Veolia Waste Centre Clyde,NaN
15452,20/01/21,14-18 Lee St,NaN,Paper/Cardboard,Mobile bin,0.24,1,NaN,Grima Recycling,NaN
20470,16/01/21,14-18 Lee St,NaN,General Waste,Mobile bin,0.24,2,50.4,Veolia Waste Centre Clyde,NaN
25106,13/01/21,14-18 Lee St,NaN,General Waste,Mobile bin,0.24,2,50.4,Veolia Waste Centre Clyde,NaN
25107,13/01/21,14-18 Lee St,NaN,Comingle,Mobile bin,0.24,1,14.4,Visy Recycling,NaN
